In [9]:
#%pip install streamlink
#%pip install ffmpeg
#to avoid:
#ImportError: cannot import name 'storage' from 'google.cloud' (unknown location)
#%pip install --upgrade google-cloud-storage

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.8/525.8 KB 5.1 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 KB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 6.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 5.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 467.2/467.2 KB 5.1 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 KB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 KB 4.5 MB/s eta 0:00:00
  Attempting uninstall: attrs
    Found existing installation: attrs 23.1.0
    Uninstalling attrs-23.1.0:
      Successfully uninstalled attrs-23.1.0
Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
from google.oauth2 import service_account

# Path to your service account JSON file
service_account_file = 'master-magnet-414308-23a8bbb0c34f.json'

# Load credentials from the JSON file
credentials = service_account.Credentials.from_service_account_file(service_account_file)

# Now you can use the credentials to authenticate with GCP services
# For example, using the BigQuery client library:
#from google.cloud import bigquery

#client = bigquery.Client(credentials=credentials, project='your-project-id')
# Use the client to run BigQuery queries or other GCP operations
credentials

In [ ]:
from google.cloud import storage

# Path to your service account JSON file
service_account_file = 'master-magnet-414308-23a8bbb0c34f.json'

# Initialize the storage client with service account credentials
storage_client = storage.Client.from_service_account_json(service_account_file)

In [6]:
from google.cloud import storage

service_account_file = 'master-magnet-414308-23a8bbb0c34f.json'
storage_client = storage.Client.from_service_account_json(service_account_file)

def upload_to_bucket(blob_name, content, bucket_name):
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(blob_name)
    blob.upload_from_filename(content)

#upload_to_bucket('link.txt', 'https://www.twitch.tv/aa', 'dl_cnn_a3_3dprintfail')

In [7]:
def upload_to_bucket(blob_name, content, bucket_name):
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(blob_name)
    blob.upload_from_filename(content)
upload_to_bucket('infer_image.jpg', 'infer_image.jpg', 'dl_cnn_a3_3dprintfail')

# ready

In [12]:
import requests
from random import choice

def fetch_content_from_gcs(link):
    alp = list(map(chr, range(97, 123)))
    link = link + "?"+choice(alp)+choice(alp)+'='+choice(alp)+choice(alp)
    try:
        response = requests.get(link)
        if response.status_code == 200:
            return response.text
        else:
            return f"Error fetching content. Status code: {response.status_code}"
    except requests.RequestException as e:
        return f"Error: {e}"

gcs_link = "https://storage.googleapis.com/dl_cnn_a3_3dprintfail/link.txt"
content = fetch_content_from_gcs(gcs_link)
#gcs_link
content

'https://www.twitch.tv/skmojotv'

In [18]:
import streamlink
import cv2
from matplotlib import pyplot as plt
import requests

def fetch_content_from_gcs(link):
    try:
        response = requests.get(link)
        if response.status_code == 200:
            return response.text
        else:
            return f"Error fetching content. Status code: {response.status_code}"
    except requests.RequestException as e:
        return f"Error: {e}"

gcs_link = "https://storage.googleapis.com/dl_cnn_a3_3dprintfail/link.txt"
content = fetch_content_from_gcs(gcs_link)
content
streams = streamlink.streams(content)
print(streams.keys())
if "best" in streams.keys():
    stream_url = streams["best"].url
    streams.keys()
else:
    print('no')
    #continue

dict_keys([])
no


In [ ]:
vcap = cv2.VideoCapture(stream_url)
image = vcap.read()[1]
cv2.imwrite('output_image.jpg', image)

plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
plt.show()

In [ ]:
import os
import subprocess

vcap = cv2.VideoCapture(stream_url)
image = vcap.read()[1]
cv2.imwrite('output_image.jpg', image)

subprocess.run(['python3', 'ObjectDetection-FasterRCNN-master/inference.py','--input','./output_image.jpg','--weights','./best_model.pth'])

foo = len(os.listdir('outputs/inference'))
#'./outputs/inference/res_{foo}/output_image.jpg'
image = cv2.imread(f'./outputs/inference/res_{foo}/output_image.jpg', cv2.IMREAD_COLOR)
cv2.imwrite('infer_image.jpg', image)
plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
plt.show()

In [ ]:
import os
import subprocess
import requests
from random import choice
import cv2
import streamlink

def fetch_content_from_gcs(link):
    alp = list(map(chr, range(97, 123)))
    link = link + "?"+choice(alp)+choice(alp)+'='+choice(alp)+choice(alp)
    try:
        response = requests.get(link)
        if response.status_code == 200:
            return response.text
        else:
            return f"Error fetching content. Status code: {response.status_code}"
    except requests.RequestException as e:
        return f"Error: {e}"

def upload_to_bucket(blob_name, content, bucket_name):
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(blob_name)
    blob.upload_from_filename(content)

gcs_link = "https://storage.googleapis.com/dl_cnn_a3_3dprintfail/link.txt"
while True:
    content = fetch_content_from_gcs(gcs_link)
    streams = streamlink.streams(content)
    if "best" not in streams.keys():
        continue
    
    stream_url = streams["best"].url
        
    vcap = cv2.VideoCapture(stream_url)
    image = vcap.read()[1]
    cv2.imwrite('output_image.jpg', image)

    #change this to be an in-process function where the model can be kept in memory, ideally yolo for faster inferencing
    subprocess.run(['python3', 'ObjectDetection-FasterRCNN-master/inference.py','--input','./output_image.jpg','--weights','./best_model.pth'])
    
    foo = len(os.listdir('outputs/inference'))
    upload_to_bucket('infer_image.jpg', f'./outputs/inference/res_{foo}/output_image.jpg', 'dl_cnn_a3_3dprintfail')
    #image = cv2.imread(f'./outputs/inference/res_{foo}/output_image.jpg', cv2.IMREAD_COLOR)
    #cv2.imwrite('infer_image.jpg', image)

Building from model name arguments...
Test instances: 1
Image 1 done...
--------------------------------------------------
TEST PREDICTIONS COMPLETE
Average FPS: 0.206
Building from model name arguments...
Test instances: 1
Image 1 done...
--------------------------------------------------
TEST PREDICTIONS COMPLETE
Average FPS: 0.296
Building from model name arguments...
Test instances: 1
Image 1 done...
--------------------------------------------------
TEST PREDICTIONS COMPLETE
Average FPS: 0.309
Building from model name arguments...
Test instances: 1
Image 1 done...
--------------------------------------------------
TEST PREDICTIONS COMPLETE
Average FPS: 0.211
Building from model name arguments...
Test instances: 1
Image 1 done...
--------------------------------------------------
TEST PREDICTIONS COMPLETE
Average FPS: 0.325
Building from model name arguments...
Test instances: 1
Image 1 done...
--------------------------------------------------
TEST PREDICTIONS COMPLETE
Average FPS

/home/user/.local/lib/python3.10/site-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,


Building from model name arguments...
Test instances: 1
Image 1 done...
--------------------------------------------------
TEST PREDICTIONS COMPLETE
Average FPS: 0.190


/home/user/.local/lib/python3.10/site-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,


Building from model name arguments...
Test instances: 1
Image 1 done...
--------------------------------------------------
TEST PREDICTIONS COMPLETE
Average FPS: 0.341


...

Building from model name arguments...
Test instances: 1
Image 1 done...
--------------------------------------------------
TEST PREDICTIONS COMPLETE
Average FPS: 0.312


/home/user/.local/lib/python3.10/site-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,


Building from model name arguments...
Test instances: 1
Image 1 done...
--------------------------------------------------
TEST PREDICTIONS COMPLETE
Average FPS: 0.224


/home/user/.local/lib/python3.10/site-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,


Building from model name arguments...
Test instances: 1
Image 1 done...
--------------------------------------------------
TEST PREDICTIONS COMPLETE
Average FPS: 0.330
Building from model name arguments...
Test instances: 1
Image 1 done...
--------------------------------------------------
TEST PREDICTIONS COMPLETE
Average FPS: 0.209
Building from model name arguments...
Test instances: 1
Image 1 done...
--------------------------------------------------
TEST PREDICTIONS COMPLETE
Average FPS: 0.320
Building from model name arguments...
Test instances: 1
Image 1 done...
--------------------------------------------------
TEST PREDICTIONS COMPLETE
Average FPS: 0.254
Building from model name arguments...
Test instances: 1
Image 1 done...
--------------------------------------------------
TEST PREDICTIONS COMPLETE
Average FPS: 0.257
Building from model name arguments...
Test instances: 1
Image 1 done...
--------------------------------------------------
TEST PREDICTIONS COMPLETE
Average FPS